In [ ]:
pip nstall numpy
pip install plotly
pip install seaborn
pip install matplotlib
pip install wordcloud
pip install textblob
pip install wordcloud

In [1]:
import json
import csv
import pandas as pd
import numpy as np
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
from textblob import TextBlob
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [3]:
# load the stored json data

items = [json.loads(f) for f in open('tweets.json', 'r')]

In [4]:
#filter the tweets for election based
tweets = []
for item in items:
    if not ("Awards" or "MAMA" or "AMA" or "BTS" or "korean" or "award" or "daesang" or "theguitarmagawards2022") in item['data']['text']:
        tweets.append(item['data']['text'])

type(tweets)

list

In [5]:
import pickle

with open('tweets.pkl', 'wb') as f:
    pickle.dump(tweets, f)

In [6]:
df = pd.read_pickle('tweets.pkl')
df = pd.DataFrame(df, columns =['text'])
id = []
id.extend(range(1, 1042806))
df['id'] = id
df

,text,id
0,@Commonsensetwet @the2ndcee @Cjsbest1 @davramd...,1
1,Read our update on the EU-US Data Transfer Fra...,2
2,@JoeBiden https://t.co/RgI1YxAS2O,3
3,RT @duty2warn: President Biden: “What makes us...,4
4,"RT @AKA_RealDirty: Biden: ""A lightning storm t...",5
...,...,...
1042800,Biden will get worse that much we can depend o...,1042801
1042801,RT @TheLeoTerrell: Gavin Newsom will run for P...,1042802
1042802,RT @JoeMcJoe11: @PoliticsVerse Cannon has made...,1042803
1042803,RT @daniels_3rd: @OccupyDemocrats What about t...,1042804


In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/athiranirmal/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/athiranirmal/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/athiranirmal/nltk_data...
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/athiranirmal/nltk_data...


True

In [10]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [11]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['text']
    myid = row['id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/1042805 [00:00<?, ?it/s]

In [12]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
vaders = vaders.merge(df, how='left')

In [13]:
vaders

,id,neg,neu,pos,compound,text
0,1,0.037,0.963,0.000,-0.2960,@Commonsensetwet @the2ndcee @Cjsbest1 @davramd...
1,2,0.000,1.000,0.000,0.0000,Read our update on the EU-US Data Transfer Fra...
2,3,0.000,1.000,0.000,0.0000,@JoeBiden https://t.co/RgI1YxAS2O
3,4,0.217,0.686,0.097,-0.5106,RT @duty2warn: President Biden: “What makes us...
4,5,0.000,1.000,0.000,0.0000,"RT @AKA_RealDirty: Biden: ""A lightning storm t..."
...,...,...,...,...,...,...
1042800,1042801,0.084,0.703,0.214,0.5859,Biden will get worse that much we can depend o...
1042801,1042802,0.229,0.685,0.086,-0.6369,RT @TheLeoTerrell: Gavin Newsom will run for P...
1042802,1042803,0.000,0.884,0.116,0.4391,RT @JoeMcJoe11: @PoliticsVerse Cannon has made...
1042803,1042804,0.000,1.000,0.000,0.0000,RT @daniels_3rd: @OccupyDemocrats What about t...


In [ ]:
bar = sns.barplot(data=vaders, x='id', y='compound')
bar.set_title('Compound Score for overall tweets')
plt.show()